In [47]:
import cv2
import os
import numpy as np

single_data = 1
multi_data = 5
more_data = 20
even_more_data = 40
full_data = 500

current_data = more_data

kmeans1_most_freq_pixel = {}

# Dataset Path
dataset_path = "../dataset/train/"

# Padding Removed Directory
dir_padding_removed = "padding_removed"
# Image Processed Directory
dir_preprocessed = "preprocessed"
# HSV Converted Directory
dir_hsv_converted = "hsv_converted"
# K Means Segmented Directory
dir_kmeans_segmented = "kmeans_segmented"
# Processed Foregrounds Directory
dir_processed_foregrounds = "processed_foregrounds"
# Resulting Detected Objects
dir_result = "detected_objectsv2"

# Remove all white pixels

In [48]:
def blackenPads(image):
    image[np.where((image==[255,255,255]).all(axis=2))] = [0,0,0]
    return image

# Simple Remove White Padding Code

In [49]:
# Remove White Padding - Gabriel Thien - Computer Visionaries

def removePads(image):
    num_rows = image.shape[0]
    num_cols = image.shape[1]
    
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(grayscale, 240, 255, cv2.THRESH_BINARY)
    
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    corners = [(0, 0) for i in range(2)]
    padding_exists = False
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        # print(f"Contour 1: {(x, y)} -> {(x + w, y + h)}")
        if (x == 0 and y == 0):
            if (x + w == num_cols): corners[0] = (x, y + h) # For vertical padding
            else: corners[0] = (x + w, y) # For horizontal padding
            padding_exists = True
        elif (x + w >= num_cols and y + h >= num_rows):
            if (y == 0): corners[1] = (x, y + h) # For vertical padding
            else: corners[1] = (x + w, y) # For horizontal padding
            padding_exists = True

    if padding_exists:
        # Displays the targeted image
        top_left_corner = corners[0]
        bottom_right_corner = corners[1]
        # cv2.rectangle(image, top_left_corner, bottom_right_corner, (0, 255, 0), 2)  # Green rectangle
        # cv2.imshow("Image Targeted", image)
    
        cropped_image = image[top_left_corner[1]:bottom_right_corner[1], top_left_corner[0]:bottom_right_corner[0]]
        # cv2.imshow("Cropped Image", cropped_image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        # cv2.waitKey(1)
        return cropped_image
    else: return image

input_directory_path = dataset_path
output_directory_path = dir_padding_removed

if os.path.exists(output_directory_path) and os.path.isdir(output_directory_path):
    pass
else: 
    os.mkdir(output_directory_path)

# Change this to process more images
for i in range(current_data):
    image_path = input_directory_path + f"image_id_{i:03d}.jpg"
    training_image = cv2.imread(image_path)
    # output = removePads(training_image)
    output = blackenPads(training_image)
    cv2.imwrite(output_directory_path + f"/result{i:03d}.png", output)

# Image Preprocessing

In [50]:
input_directory_path = dir_padding_removed
output_directory_path = dir_preprocessed

if os.path.exists(output_directory_path) and os.path.isdir(output_directory_path):
    pass
else: 
    os.mkdir(output_directory_path)

def sharpen(image, iterations=1):
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])

    for i in range(iterations):
        image = cv2.filter2D(image, -1, kernel)

    return image

def changeSaturation(image, saturation_factor):
    # Convert BGR image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Split the HSV image into separate channels
    h, s, v = cv2.split(hsv_image)

    # Increase saturation (clip to ensure valid range of [0, 255])
    s = np.clip(s * saturation_factor, 0, 255).astype(np.uint8)

    # Merge the modified saturation channel back with the other channels
    hsv_adjusted = cv2.merge((h, s, v))

    # Convert the HSV-adjusted image back to BGR color space
    adjusted_image = cv2.cvtColor(hsv_adjusted, cv2.COLOR_HSV2BGR)

    return adjusted_image

saturation_factor = 4

for i in range(current_data):
    imagePath = input_directory_path + f"/result{i:03d}.png"
    input_image = cv2.imread(imagePath)
    
    sharper = sharpen(input_image)

    saturated = changeSaturation(sharper, saturation_factor)

    # cv2.imshow("Contrasted", contrasted)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # cv2.waitKey(1)
    
    cv2.imwrite(output_directory_path + f"/result{i:03d}.png", saturated)    

# HSV Generator
- Output: (hsv_padding_removed)

In [51]:
# Blurring & Generating HSVs - Gabriel Thien - Computer Visionaries

input_directory_path = dir_preprocessed
output_directory_path = dir_hsv_converted

if os.path.exists(output_directory_path) and os.path.isdir(output_directory_path):
    pass
else: 
    os.mkdir(output_directory_path)

for i in range(current_data):
    imagePath = input_directory_path + f"/result{i:03d}.png"
    input_image = cv2.imread(imagePath)
    hsv_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2HSV)
    
    # Blurring
    kernel_size_mean = (3, 3)
    hsv_blurred = cv2.blur(hsv_image, kernel_size_mean)
    
    # cv2.imshow("Blurrer HSV", hsv_blurred)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    # cv.waitKey(1)
    
    cv2.imwrite(output_directory_path + f"/result{i:03d}.png", hsv_blurred)

# K Means Segmented
- Output: hsv_kmeans_segmented

In [52]:
# Colour Segmentation - Gabriel Thien - Computer Visionaries

from sklearn.cluster import KMeans

def makeFlattenedMatrix(image):
    pixels = []
    
    for row in image:
        for rgb_pixel in row:
            pixels.append((rgb_pixel[0], rgb_pixel[1], rgb_pixel[2]))

    return np.array(pixels)    
    

def kMeansSegment(imagePath):
    image = cv2.imread(imagePath)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    pixels = makeFlattenedMatrix(image_rgb)

    # K means clustering
    K = 4
    n = 2
    # Perform K-means clustering
    kmeans = KMeans(n_init=n, n_clusters=K, random_state=0)
    kmeans.fit(pixels)

    cluster_centers = kmeans.cluster_centers_
    labels = kmeans.labels_
    # print(cluster_centers)
    # print(labels)

    # Create the segmented image
    segmented_image = cluster_centers[labels].reshape(image.shape).astype(np.uint8)
    
    return segmented_image

# cv2.waitKey(0)
# cv2.destroyAllWindows()
# cv2.waitKey(1)

input_directory_path = dir_hsv_converted
output_directory_path = dir_kmeans_segmented

if os.path.exists(output_directory_path) and os.path.isdir(output_directory_path):
    pass
else: 
    os.mkdir(output_directory_path)

for i in range(current_data):
    imagePath = input_directory_path + f"/result{i:03d}.png"
    result = kMeansSegment(imagePath)
    cv2.imwrite(output_directory_path + f"/result{i:03d}.png", result)

# Determining colour frequencies of a segmented image

In [53]:
# Computing Colours - Gabriel Thien - Computer Visionaries

def computePixelFrequencies(image) -> list:
    pixels = {}
    for row in image:
        for pixel in row:
            if tuple(pixel) in pixels.keys():
                pixels[tuple(pixel)] += 1
            else:
                pixels[tuple(pixel)] = 1

    return sorted(pixels.items(), key=lambda x: x[1])


def isolateColour(image, colours: list):

    # black_image = np.zeros((num_rows, num_cols, 3), dtype=np.uint8)

    image_copy = image.copy()
    pure_colours = [colours[i][0] for i in range(len(colours))]

    # Colours - FOUR
    # colour_low = tuple(colours[0][0])
    # colour_mid = tuple(colours[1][0])
    # colour_high = tuple(colours[2][0])
    # colour_max = tuple(colours[3][0])
    
    # Colours - THREE
    # colour_low = tuple(colours[0][0])
    # colour_mid = tuple(colours[1][0])
    # colour_high = tuple(colours[2][0])
    # print(colour_low)

    # Colours - TWO
    # colour_low = tuple(colours[0][0])
    # colour_high = tuple(colours[1][0])

    # Redo Colours
    for i in range(len(colours)):
        pixel = colours[i][0]
        if (pixel[0] <= 20 and pixel[1] <= 20 and pixel[2] <= 20):
            colours.remove(colours[i])
            break
    
    colour_low = tuple(colours[0][0])
    colour_mid = tuple(colours[1][0])
    colour_high = tuple(colours[2][0])

    # K1
    kernel = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]], dtype=np.uint8)

    # K2
    # kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]], dtype=np.uint8)

    # K3
    # kernel = np.ones((3, 3), np.uint8)

    # Original
    # cv2.imshow("Original", image)

    # Foreground stuff
    # foreground_colour = preserveSingleColour(image_copy, colour_low)
    foreground = preserveDoubleColour(image_copy, colour_low, colour_mid)
    foreground = cv2.erode(foreground, kernel, iterations=10)
    foreground = cv2.dilate(foreground, kernel, iterations=4)

    # # Midground stuff
    # midground_colour = preserveSingleColour(image_copy, colour_mid)

    # Background stuff
    # background_colour = preserveSingleColour(image_copy, colour_high)
    # background_dilated = cv2.dilate(background_colour, kernel, iterations=20)
    # background_eroded = cv2.erode(background_dilated, kernel, iterations=20)
    # background_focused_image = cv2.bitwise_not(background_eroded)

    # cv2.imshow("Foreground Colour", foreground_colour)
    # cv2.imshow("Eroded", foreground_eroded)
    # cv2.imshow("Foreground", foreground_focused_image)

    # cv2.imshow("Background Colour", background_colour)
    # cv2.imshow("Background Dilated", background_dilated)
    # cv2.imshow("Background Eroded", background_eroded)
    # cv2.imshow("Background", background_focused_image)

    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # cv2.waitKey(1)
    return foreground


def preserveSingleColour(image, targetPixel: tuple):
    image_copy = image.copy()

    # Pixel mask that isnt the target pixel
    mask = np.all(image_copy != targetPixel, axis=2)
    # Applies mask
    image_copy[mask] = [0, 0, 0]

    # cv2.imshow("Grayscale single channel", grayscale)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # cv2.waitKey(1)
    return image_copy


def preserveDoubleColour(image, targetPixel1: tuple, targetPixel2: tuple):
    foreground = image.copy()
    midground = image.copy()

    # Pixel mask that isnt the target pixel
    foreground_mask = np.all(foreground != targetPixel1, axis=2)
    mid_ground_mask = np.all(midground != targetPixel2, axis=2)
    # Applies mask
    foreground[foreground_mask] = [0, 0, 0]
    midground[mid_ground_mask] = [0, 0, 0]
    result = cv2.add(foreground, midground)

    # cv2.imshow("Grayscale single channel", grayscale)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # cv2.waitKey(1)
    return result


def defaultBackground(image, colours: tuple, tolerance=20):
    num_rows = image.shape[0]
    num_cols = image.shape[1]

    most_common_colour = colours[-1][0]

    for x in range(num_rows):
        for y in range(num_cols):
            pixel = image[x][y]
            if (
                pixel[0] <= tolerance
                and pixel[1] <= tolerance
                and pixel[2] <= tolerance
            ):
                image[x][y] = most_common_colour

    return image

def removeBackground(image, colours: tuple):
    num_rows = image.shape[0]
    num_cols = image.shape[1]

    most_common_colour = colours[-1][0]

    for x in range(num_rows):
        for y in range(num_cols):
            pixel = image[x][y]
            if tuple(pixel) == most_common_colour:
                image[x][y] = (0, 0, 0)

    colours.pop()
    
    return image, colours


input_directory_path = dir_kmeans_segmented
output_directory_path = dir_processed_foregrounds

if os.path.exists(output_directory_path) and os.path.isdir(output_directory_path):
    pass
else: 
    os.mkdir(output_directory_path)

for i in range(current_data):
    imagePath = input_directory_path + f"/result{i:03d}.png"
    image = cv2.imread(imagePath)

    pixel_frequencies = computePixelFrequencies(image)
    # print(f"Number of pixel groups for image {i}: {len(pixel_frequencies)}, {pixel_frequencies}")

    most_frequent_pixel = pixel_frequencies[-1][0]
    kmeans1_most_freq_pixel[i] = most_frequent_pixel

    defaulted_background = defaultBackground(image, pixel_frequencies)

    # Isolate foreground based on pixel frequencies
    processed_image = isolateColour(defaulted_background, pixel_frequencies)

    cv2.imwrite(output_directory_path + f"/result{i:03d}.png", processed_image)

    # cv2.imshow(f"Processed Image", processed_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # cv2.waitKey(1)

# Drawing Bounding Boxes

In [54]:
# Draw Bounding Boxes - Gabriel Thien - Computer Visionaries
# Image has been segmented and the outline of the image is ready to be highlighted

def getContours(image):
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    _, thresholded = cv2.threshold(grayscale, 0, 255, cv2.THRESH_BINARY)
    
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # print(f"Type of contours: {type(contours)}")
    return contours

# All areas
def calculateBoundingBox(contours):
    x_min = x_max = 0
    y_min = y_max = 0
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if (x_min == 0) and (x_max == 0) and (y_min == 0) and (y_max == 0):
            x_min = x
            x_max = x + w
            y_min = y
            y_max = y + h
        if (x < x_min): x_min = x
        elif (x + w > x_max): x_max = x + w
        if (y < y_min): y_min = y
        elif (y + h > y_max): y_max = y + h
        
    return (x_min, y_min), (x_max, y_max)

def drawBoundingBox(island_image, original_image):
    contours = getContours(island_image)

    top_left_corner, bottom_right_corner = calculateBoundingBox(contours)

    drawn = cv2.rectangle(original_image, top_left_corner, bottom_right_corner, (0, 255, 0), 2)
    return drawn

input_directory_path = dir_processed_foregrounds
output_directory_path = dir_result

if os.path.exists(output_directory_path) and os.path.isdir(output_directory_path):
    pass
else: 
    os.mkdir(output_directory_path)

for i in range(current_data):
    islandImagePath = input_directory_path + f"/result{i:03d}.png"
    island_image = cv2.imread(islandImagePath)
    
    originalImagePath = dataset_path + f"image_id_{i:03d}.jpg"
    original_image = cv2.imread(originalImagePath)
    
    drawn_image = drawBoundingBox(island_image, original_image)
        
    cv2.imwrite(output_directory_path + f"/detectedv2_{i:03d}.png", drawn_image)
    # cv2.imshow("Drawn Image", drawn_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # cv2.waitKey(1)